In [79]:
from datetime import datetime
import pandas as pd
from geopy import distance
import numpy as np

import random

import pymongo
from pymongo import MongoClient

In [80]:
shapes = pd.read_csv('sfmta_2017-02-10/shapes.txt')
trips = pd.read_csv('sfmta_2017-02-10/trips.txt')
sched = pd.read_csv('stop_sequence.csv')
stops = pd.read_csv('sfmta_2017-02-10/stops.txt')

In [81]:
stops.head()

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url
0,98,2ND ST & MARKET ST,,37.789255,-122.401225,,
1,100,STEUART ST & FRANCISCO ST,,37.794463,-122.394607,,
2,202,COLUMBUS AVE & Pacific Ave,,37.797055,-122.405669,,
3,210,LINCOLN WAY & 19TH AVE,,37.765375,-122.477197,,
4,383,19TH AVE & LINCOLN WAY,,37.765375,-122.477197,,


In [82]:
client = MongoClient('localhost', 27017)

db = client['avl_pipeline_test']
trip_coll = db['trips']

In [83]:
sample_id = trip_coll.find_one({},{'_id':0, 'trip_id':1})['trip_id']
shape_id = trips[trips['trip_id'] == sample_id]['shape_id'].values[0]
trip_shape = shapes[shapes['shape_id'] == shape_id]

In [84]:
def get_dist(row, stop_tuple):
    shape_lat = float(row['shape_pt_lat'])
    shape_lon = float(row['shape_pt_lon'])
    return distance.distance((shape_lat, shape_lon), stop_tuple).m

In [85]:
stops_distances = []
stop_lat = []
stop_lon = []
stop_name = []

for idx, stp_srs in sched.iterrows():
    
    # Get the details of the stops
    stp_id = stp_srs['stop_id']
    stp_dtls = stops[stops['stop_id'] == stp_id]
    
    stp_lat = float(stp_dtls['stop_lat'])
    stp_lon = float(stp_dtls['stop_lon'])
    
    stop_lat.append(stp_lat)
    stop_lon.append(stp_lon)
    
    stop_name.append(stp_dtls['stop_name'].values[0])
    
    tups = (stp_lat, stp_lon)
    
    trip_shape['stop_dist'] = trip_shape.apply(lambda row: get_dist(row, tups), axis=1)
    
    closest_idx = trip_shape['stop_dist'].idxmin()
    
    stops_distances.append(trip_shape.loc[closest_idx]['shape_dist_traveled'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [86]:
sched['stop_distance'] = stops_distances
sched['stop_lat'] = stop_lat
sched['stop_lon'] = stop_lon
sched['stop_name'] = stop_name

In [87]:
stop_name

['Sacramento St & Cherry St',
 'California St & Maple St',
 'California St & Cherry St',
 'Arguello Blvd & California St',
 'Arguello Blvd & Clement St',
 'Arguello Blvd & Geary Blvd',
 'Arguello Blvd & Balboa St',
 'Fulton St & Arguello Blvd',
 'Stanyan St & Fulton St',
 'Stanyan St & Hayes St',
 'Haight St & Stanyan St',
 'Haight St & Clayton St',
 'Ashbury St & Waller St',
 'Ashbury St & Frederick St',
 'Ashbury St & Piedmont St',
 'Ashbury St & Clifford Ter',
 'ASHBURY ST & CLAYTON ST',
 'Clayton St & Twin Peaks Blvd',
 'Clayton St & Corbett Ave',
 'Clayton St & Market St',
 'Market St & Clayton St',
 '18th St & Danvers St',
 '18th St & Hattie St',
 '18th St & Diamond St',
 '18th St & Castro St',
 '18th St & Noe St',
 '18th St & Sanchez St',
 '18th St & Church St',
 '18th St & Dolores St',
 '18th St & Guerrero St',
 '18th St & Valencia St',
 'Mission St & 18th St',
 '16th St & Mission St',
 '16th St & Shotwell St',
 '16th St & Harrison St',
 '16th St & Bryant St',
 '16th St & Potre

In [88]:
sched

,stop_id,stop_sequence,seq_str,stop_distance,stop_lat,stop_lon,stop_name
0,6293,1,1,0.0,37.786908,-122.456560,Sacramento St & Cherry St
1,3879,2,2,206.0,37.786248,-122.455207,California St & Maple St
2,3852,3,3,352.0,37.786042,-122.456835,California St & Cherry St
3,3644,4,4,575.0,37.785560,-122.459254,Arguello Blvd & California St
4,3645,5,5,852.0,37.783071,-122.459070,Arguello Blvd & Clement St
5,3649,6,6,1067.0,37.781429,-122.458955,Arguello Blvd & Geary Blvd
6,3642,7,7,1513.0,37.777075,-122.458645,Arguello Blvd & Balboa St
7,4224,8,8,1842.0,37.774282,-122.458003,Fulton St & Arguello Blvd
8,6479,9,9,2184.0,37.774604,-122.454714,Stanyan St & Fulton St
9,6481,10,10,2381.0,37.772766,-122.454335,Stanyan St & Hayes St


In [89]:
sched.to_csv('stop_seq_dist_latlon.csv', index=False)